# DIMS.FARM REST API example

In [1]:
import pandas as pd
import requests
import json
import pandas.io.json as pj

In [6]:
API_URL = "http://10.101.0.29:8099/api"
TOKEN = ""

In [7]:
endpoint = "/Login/Login"
r = requests.post(API_URL + endpoint, params={"TOKEN": TOKEN}) 
r.raise_for_status()
r.json()

{'session': '{712A64D2-3320-4BF1-82E8-FE439D99C181}'}

In [8]:
session = r.json()["session"]
session

'{712A64D2-3320-4BF1-82E8-FE439D99C181}'

In [11]:
endpoint = "/TimeSeries/List"
r = requests.post(API_URL + endpoint, headers={"Access-Token": session}) 
r.raise_for_status()
data = r.json()

In [32]:
cols = [f["name"] for f in data["schema"]["fields"]]

In [21]:
def ReadFarmPandaJson(json_input):
    ac = [] # Column names
    o = json.loads(json_input)
    for f in o["schema"]["fields"]:
        ac.append(f["name"])
    df = pd.read_json(json.dumps(o["data"]), orient="values") # Create the dataframe from values - index is being created automatically, that should be ignored!
    df.columns = ac # Rename columns to correct names
    zipped = zip(o["schema"]["fields"], pj.build_table_schema(df, index=False)["fields"]) # ignoring index from the newly created frame
    for m, l in zipped:
        if m["type"] != l["type"]:
            if m["type"] == "datetime":
                df.loc[:,m["name"]] = pd.to_datetime(df.loc[:,m["name"]], unit='ms')
            if m["type"] == "number":    
                df.loc[:,m["name"]] = df.loc[:,m["name"]].astype(float)
    df.set_index(o["schema"]["primaryKey"][0], inplace=True) # Setting index in actual dataframe, Assume index name from PK
    return df

In [33]:
df = ReadFarmPandaJson(r.text)
df.head()

,EntityID,Touched,IDName,IDDescription,LocationID,TimeSeriesParameterID,TimeSeriesMediaID,TimeSeriesUnitID,TimeSeriesDataTypeID,TimeSeriesDatasourceID,TimeSeriesTypeID,TimeSeriesStatusID,TimeSeriesDataProviderID,TimeSeriesDataProviderArgument,DataExpectedCount,DataArchiveAfter,DataDeleteAfter
ID,,,,,,,,,,,,,,,,,
2,BIRKUD.Intern niveau.Offset.dvr,2011-10-25 09:52:07,BIRKUD.Intern niveau.Offset.dvr,Niveau offset dnn målt Birkedalen til Streget(...,BIRKUD,Niveau offset dvr,Vandløb,m,Unknown,SIGNALIX,Offset,,,,NaN,NaN,NaN
3,BIRKUD.Intern niveau.dvr,2011-10-25 09:52:08,BIRKUD.Intern niveau.dvr,Niveau dnn målt Birkedalen til Streget(st. 1354),BIRKUD,Niveau dvr,Vandløb,m,Unknown,SIGNALIX,Beregnet,,,,NaN,NaN,NaN
4,BIRKUD.Intern niveau,2016-09-22 09:48:56,BIRKUD.Intern niveau,Niveau målt Birkedalen til Streget(st. 1354),BIRKUD,Niveau,Vandløb,m,Unknown,SIGNALIX,Datalogger,,,,NaN,NaN,NaN
5,BIRKUD.Hastigheden,2016-09-22 09:48:57,BIRKUD.Hastigheden,Hastighed målt Birkedalen til Streget(st. 1354),BIRKUD,Hastighed,Vandløb,m/s,Unknown,SIGNALIX,Datalogger,,,,NaN,NaN,NaN
6,BIRKUD.Flow_korrigeret,2012-01-29 18:56:42,BIRKUD.Flow_korrigeret,Flow målt Birkedalen til Streget(st. 1354),BIRKUD,Flow Korrigeret,Vandløb,l/s,Unknown,SIGNALIX,Beregnet,,,,NaN,NaN,NaN
